fig1c: milo cell type abundance analaysis

# Import and settings

In [1]:
%load_ext autoreload
%autoreload 2
%load_ext rmagic

import os, re, gc, joblib
from pathlib import Path
import numpy as np
import numpy_groupies as npg
import pandas as pd

import anndata
import scanpy as sc
import sctk as sk

import matplotlib.pyplot as plt
from matplotlib import rcParams
from matplotlib.backends.backend_pdf import PdfPages

ModuleNotFoundError: No module named 'rmagic'

In [3]:
np.set_printoptions(linewidth=150)

rcParams["pdf.fonttype"] = 42
rcParams["ps.fonttype"] = 42

sc.settings.verbosity = 0
sk.set_figsize((4, 4))

In [4]:
from time import localtime, strftime


def timestamp(date_only=True):
    stamp_format = "%Y%m%d" if date_only else "%Y%m%d-%H%M"
    return strftime(stamp_format, localtime())

In [5]:
os.environ["R_HOME"] = "/home/ubuntu/miniconda/envs/milo/lib/R"

In [6]:
data_root = Path('/lustre/scratch126/cellgen/team205/nh3/skin').resolve()
proj_root = Path('~/FCA_Fetal_Skin_priv/').expanduser()
os.chdir(proj_root)
!pwd

/home/jovyan/FCA_Fetal_Skin_priv


# Load data

In [7]:
fsk_all_obs = sk.read_h5ad(Path(data_root, 
                                "20210611_final_object/fetal_skin.norm.maternal_removed.20220202.h5ad"), 
                           component="obs")

In [8]:
fsk_all_obs.columns

Index(['sanger_id', 'chemistry', 'donor', 'gender', 'pcw', 'sorting', 'sample',
       'chemistry_sorting', 'cell_caller', 'scrublet_score',
       'cluster_scrublet_score', 'doublet_pval', 'bh_doublet_pval', 'n_counts',
       'n_genes', 'percent_mito', 'percent_ribo', 'percent_hb',
       'independent_annotation_refined', 'independent_annotation_broad1',
       'independent_annotation_broad2', 'independent_annotation_broad3',
       'independent_annotation_broad4', 'independent_annotation_broad5',
       'joint_annotation', 'is_maternal', 'fig1b_annotation',
       'independent_annotation_refined_20220201a',
       'independent_annotation_refined_20220201b',
       'independent_annotation_broad1_20220201',
       'independent_annotation_broad2_20220201',
       'independent_annotation_broad3_20220201', 'joint_annotation_20220201a',
       'joint_annotation_20220201b', 'fig1b_annotation_20220201',
       'joint_annotation_20220202', 'independent_annotation_refined_20220202',
       'f

In [9]:
# fsk_all_obs.to_csv("../20210611_final_object/fetal_skin.norm.maternal_removed.20220202.obs.csv.gz")

In [10]:
%%R
# conda activate milo

loadPackage(miloR)
loadPackage(SingleCellExperiment)
loadPackage(scater)
loadPackage(dplyr)
loadPackage(Matrix)


# new analysis 20220202
sce <- readRDS('fetal_skin.scvi_donor.norm.sce.rds')

obs <- read.csv("../20210611_final_object/fetal_skin.norm.maternal_removed.20220202.obs.csv.gz", row.names=1)

sce <- sce[, colnames(sce) %in% rownames(obs)]

milo <- Milo(sce)
milo <- buildGraph(milo, k=15, d=30, reduced.dim = "X_scvi_donor")
milo <- makeNhoods(milo, prop=0.05, k=15, d=30, refined=T, reduced_dims='X_scvi_donor')
milo <- calcNhoodDistance(milo, d=30, reduced.dim='X_scvi_donor')
milo <- countCells(milo, meta.data=as.data.frame(colData(milo)), sample='sanger_id')
saveRDS(milo, 'fetal_skin.scvi_donor.milo.20220202.rds')

milo_design <- data.frame(colData(milo)[, c('donor', 'pcw', 'sorting', 'sanger_id')])
milo_design <- distinct(milo_design)
rownames(milo_design) <- milo_design$sanger_id

results <- testNhoods(milo, design=~sorting + pcw, design.df=milo_design, norm.method='TMM')
saveRDS(results, 'fetal_skin.scvi_donor.milo_results.20220202.rds')

## fig1b_annotation

fig1b_annotation <- obs$fig1b_annotation_20220202 %>% as.character()
fig1b_annotation[fig1b_annotation=='Progenitor'] = 'Haem progenitors'
fig1b_annotation[fig1b_annotation=='Neuronal cells'] = 'Neuronal cell'
fig1b_order <- c(
    'Haem progenitors', 'ILC', 'T cell', 'B cell', 'pDC', 'cDC', 'Langerhans cell',
    'Macrophage', 'Monocyte', 'Neutrophil', 'Mast cell', 'Megakaryocyte', 'Erythroid', 'Vascular endothelium',
    'Lymphatic endothelium', 'Mural cell', 'Skeletal muscle', 'Myofibroblast', 'Fibroblast', 'Dermal condensate',
    'Adipocyte', 'Keratinocyte', 'Melanocyte', 'Schwann cell', 'Neuronal cell'
)
colData(milo)$fig1b_annotation <- factor(fig1b_annotation, levels=fig1b_order)

colData(milo)$independent_annotation_refined <- obs$independent_annotation_refined_20220202
colData(milo)$joint_annotation <- obs$joint_annotation_20220202

ct_results <- annotateNhoods(milo, results, coldata_col='fig1b_annotation')
ct_results$nhood_annotation <- ifelse(ct_results$fig1b_annotation_fraction < 0.7, 'Mixed', ct_results$fig1b_annotation)
ordered_ct <- (ct_results %>% group_by(fig1b_annotation) %>% summarise(mean_logFC=-mean(logFC)) %>% arrange(mean_logFC))$fig1b_annotation

p1 <- (
    plotDAbeeswarm(ct_results %>% filter(nhood_annotation != 'Mixed'), group.by='nhood_annotation', alpha=0.05)
    + theme(axis.text.y=element_text(size=8))
)
p1a <- p1 + scale_x_discrete(limits=rev(fig1b_order))
q1a <- ggplot_build(p1a)
q1a$data[[1]]$size <- 0.5
gp1a <- ggplot_gtable(q1a)

p1b <- p1 + scale_x_discrete(limits=ordered_ct)
q1b <- ggplot_build(p1b)
q1b$data[[1]]$size <- 0.5
gp1b <- ggplot_gtable(q1b)

pdf(file='fetal_skin.scvi_donor.milo.fig1b_annotation_vs_pcw.20220202.pdf')
plot(gp1a)
plot(gp1b)
dev.off()

UsageError: Cell magic `%%R` not found.
